In [1]:
import requests

import pickle
from pymongo import MongoClient

from config import genius_access_token

### Connecting to MongoDB

In [4]:
client = MongoClient()
db = client.genius
db.list_collection_names()

['artist_ids']

## Import and format artist names for API call

In [5]:
pickle_in = open('../Data/artists_to_query_genius', 'rb')
artists = pickle.load(pickle_in)
pickle_in.close()

In [6]:
len(artists)

539

Replaces spaces with `%20` for API call

In [7]:
artists = [x.replace(' ', '%20') for x in artists]

## Write to mongoDB

In [11]:
def get_artist_ids(artist_names, access_token, headers):
    """
    Takes in a list of artist names, the api access token
    for genius lyrics and header information. 
    It then writes that information into a mongoDB.
    
    If no lyrics are present for a song then no entry
    is made into the database for the artist/song."""

    for artist_name in artist_names:
        artist_info = {}
        url = "http://api.genius.com/search?q={}".format(artist_name)

        resp = requests.get(url, headers=headers)
        resp.raise_for_status()
        json = resp.json()

        try:
            artist_id = (json['response']
                         ['hits'][0]
                         ['result']['primary_artist']['id'])

            artist_info['artist_name'] = artist_name.replace('%20', ' ')
            artist_info['artist_id'] = artist_id

            db.artist_ids.insert_one(artist_info)
        except:
            print('Failed for '+artist_name)

In [9]:
access_token = genius_access_token
headers = {
         "Accept": "application/json",
         "Content-Type": "application/json",
         "Authorization": "Bearer " + access_token
        }

In [12]:
get_artist_ids(artists, access_token, headers)

Failed for Chloe%20Temtchine
Failed for The%20Trolleyvox
Failed for MagellanMusic
